In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

ens='cA211.530.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
path='/project/s1174/lyan/code/projectData/NST_a/test/data_merge/data.h5_alldiags'
data=yu.load(path)
Ncfg=yu.deepKey(data['2pt'],2).shape[0]
print('Ncfg='+str(Ncfg))

In [ ]:
def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    if yu.getNpars(opa,opb)==(2,1):
        return np.conj(get2pt(opb,opa,diags))
    t=data['2pt'][opa+'_'+opb] 
    res=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['2pt'].keys(): # contibution for c.c of T-pi0i
        t=data['2pt'][opb+'_'+opa]; diag='T-pi0f'
        res+= np.conj(t[diag]) if diag in t.keys() and diag in diags else 0
    return res
def get2ptMat(ops,diags=yu.diags_all):
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    # t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    if yu.getNpars(opa,opb)==(2,1):
        return np.conj(get3pt(opb,opa,insert,diags)[:,::-1])*(1 if insert.split('_')[0] in ['id','gt'] else (-1)) # wait to check
    t=data['3pt'][opa+'_'+opb][insert]
    return np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
def get3ptMat(opas,opbs,insert,diags=yu.diags_all,NpiNpi=False):
    skipNpiNpi=lambda opa,opb: yu.getNpars(opa,opb)==(2,2) and not NpiNpi
    t=np.transpose([[get3pt(opa,opb,insert,diags) if not skipNpiNpi(opa,opb) else np.zeros([Ncfg,int(insert.split('_')[-1])+1])  for opb in opbs] for opa in opas],[2,3,0,1])
    # t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

In [ ]:
def run2pt(pt,ofa,ofb,prefactor='re',diags=yu.diags_all,title=None,ylim1=None,ylim2=None):
    fig, axs = yu.getFigAxs(1,2)
    title=title if title=='None' else 'pt={}; ofa={}; ofb={}; prefactor={}'.format(pt,ofa,ofb,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=1; tmax=15
    
    for l,color,xshift in zip(['l1','l2'],['r','b'],[-0.1,0.1]):
        opa,opb=yu.getopab(pt,l,ofa,ofb)
        dat=get2pt(opa,opb,diags)
        
        irow=0; icol=0
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=1
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            t=np.log(t/np.roll(t,-1,axis=0))
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim2)
        
# run2pt('0,0,0',('a','p'),('a','p'))
# run2pt('0,0,1',('a','p'),('a','p'))

# run2pt('0,0,0',('a','p'),('N1pi1,a','n,pi+'))
# run2pt('0,0,0',('a','p'),('N1pi1,a','p,pi0'),diags=['T'])
# run2pt('0,0,0',('a','p'),('N1pi1,a','p,pi0'),diags=['N-pi0i'])

# run2pt('0,0,1',('a','p'),('N0pi1,a','n,pi+'))
# run2pt('0,0,1',('a','p'),('N0pi1,a','n,pi+'),prefactor='im')
# run2pt('0,0,1',('a','p'),('N0pi1,a','p,pi0'))
# run2pt('0,0,1',('a','p'),('N0pi1,a','p,pi0'),prefactor='im')

# run2pt('0,0,1',('a','p'),('N1pi0,a','n,pi+'))
# run2pt('0,0,1',('a','p'),('N1pi0,a','n,pi+'),prefactor='im')

# run2pt('0,0,1',('a','p'),('N1pi0,a','p,pi0'),diags=['N-pi0i'],ylim1=[-5e-9,5e-9])
# run2pt('0,0,1',('N1pi0,a','p,pi0'),('a','p'),diags=['N-pi0f'],ylim1=[-5e-9,5e-9])
# run2pt('0,0,1',('a','p'),('N1pi0,a','p,pi0'),diags=['N-pi0i'],prefactor='im')
# run2pt('0,0,1',('N1pi0,a','p,pi0'),('a','p'),diags=['N-pi0f'],prefactor='im')

# run2pt('0,0,0',('N1pi1,a','n,pi+'),('N1pi1,a','p,pi0'))
# run2pt('0,0,0',('N1pi1,a','p,pi0'),('N1pi1,a','n,pi+'))
# run2pt('0,0,0',('N1pi1,a','p,pi0'),('N1pi1,a','p,pi0'))

# run2pt('0,0,1',('N1pi0,a','n,pi+'),('N1pi0,a','n,pi+'))

In [ ]:
def run2pt_Npi0(pt,ofa,ofb,prefactor='re',title=None,ylim1=None,ylim2=None):
    fig, axs = yu.getFigAxs(1,3)
    title=title if title=='None' else 'pt={}; ofa={}; ofb={}; prefactor={}'.format(pt,ofa,ofb,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=0; tmax=15
    
    for l,color,xshift in zip(['l1','l2'],['r','b'],[-0.1,0.1]):
        opa,opb=yu.getopab(pt,l,ofa,ofb)
        
        irow=0; icol=0
        dat=get2pt(opa,opb,['N-pi0i'])
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=1
        dat=get2pt(opb,opa,['N-pi0f'])
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=2
        dat=(get2pt(opa,opb,['N-pi0i'])+get2pt(opb,opa,['N-pi0f']))/2
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(np.array(ylim1)/5)
        axs[irow,icol].set_ylim(ylim2)

# run2pt_Npi0('0,0,0',('a','p'),('N1pi1,a','p,pi0'),ylim1=[-3e-9,1e-9],ylim2=[-3e-9,1e-9])
# run2pt_Npi0('0,0,1',('a','p'),('N0pi1,a','p,pi0'),ylim1=[-3e-9,1e-9],ylim2=[-3e-9,1e-9])
# run2pt_Npi0('0,0,1',('a','p'),('N1pi0,a','p,pi0'),ylim1=[-10e-9,10e-9],ylim2=[-1e-9,1e-9])

In [ ]:
def run2pt_cc(pt,of1,of2,prefactor='re',diags=yu.diags_all,title=None,ylim=None):
    fig, axs = yu.getFigAxs(1,1)
    # title=title if title=='None' else 'pt={}; of1={}; of2={}, prefactor={}'.format(pt,of1,of2,prefactor)
    # fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=0; tmax=15
    
    for ofs,isoFactor,color,xshift in zip([(of1,of2),(of2,of1)],[1,1 if prefactor==1 else -1],['r','b'],[-0.1,0.1]):
        opa,opb=yu.getopab(pt,'l1',ofs[0],ofs[1])
        
        irow=0; icol=0
        dat=get2pt(opa,opb,diags)
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)

ri='re'
# run2pt_cc('0,0,0',('a','p'),('N1pi1,a','p,pi0'),prefactor=ri,diags=['N-pi0i','N-pi0f'])
# run2pt_cc('0,0,1',('a','p'),('N1pi0,a','p,pi0'),prefactor=ri,diags=['N-pi0i','N-pi0f'])
# run2pt_cc('0,0,1',('a','p'),('N0pi1,a','p,pi0'),prefactor=ri,diags=['N-pi0i','N-pi0f'])
# run2pt_cc('0,0,1',('N1pi0,a','n,pi+'),('N1pi0,a','p,pi0'),prefactor=ri,diags=['W2pt'])
# run2pt_cc('0,0,1',('N1pi0,a','p,pi0'),('N0pi1,a','n,pi+'),prefactor=ri,diags=yu.diags_all)

In [ ]:
def run2pt_iso_NNpi(pt,occ,prefactor='re',title=None,ylim=None):
    fig, axs = yu.getFigAxs(1,2)
    title=title if title=='None' else 'pt={}; occ={}; prefactor={}'.format(pt,occ,prefactor)
    fig.suptitle(title)
    yu.addColHeader(axs,['loopless','all'])
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=1; tmax=15
    
    for fla,isoFactor,color,xshift in zip(['n,pi+','p,pi0'],[1,(-1*np.sqrt(2))],['r','b'],[-0.1,0.1]):
        opa,opb=yu.getopab(pt,'l1',('a','p'),(occ,fla))
        
        irow=0; icol=0
        dat=get2pt(opa,opb,yu.diags_loopless)
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)
        
        irow=0; icol=1
        dat=get2pt(opa,opb,yu.diags_all)
        if fla=='p,pi0':
            dat=get2pt(opa,opb,['T'])+get2pt(opb,opa,['N-pi0f'])
            dat=get2pt(opa,opb,['T'])+(get2pt(opb,opa,['N-pi0f'])+np.conj(get2pt(opa,opb,['N-pi0i'])))/2
            # dat=get2pt(opa,opb,['T'])+get2pt(opa,opb,['N-pi0i'])
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)

# ri='re'
# ylim=[-1e-9,1e-9]; ylim=None
# run2pt_iso_NNpi('0,0,0','N1pi1,a',prefactor=ri,ylim=ylim)
# run2pt_iso_NNpi('0,0,1','N1pi0,a',prefactor=ri,ylim=ylim)
# run2pt_iso_NNpi('0,0,1','N0pi1,a',prefactor=ri,ylim=ylim)

In [ ]:
def run2pt_iso_NpiNpi(pt,occ,prefactor='re',title=None,ylim=None):
    fig, axs = yu.getFigAxs(1,2)
    title=title if title=='None' else 'pt={}; occ={}; prefactor={}'.format(pt,occ,prefactor)
    fig.suptitle(title)
    yu.addColHeader(axs,['loopless','all'])
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=1; tmax=15
    for flas,isoFactor,color,xshift in zip([('n,pi+','n,pi+'),('p,pi0','n,pi+'),('p,pi0','p,pi0')],[1,-1*np.sqrt(2),2],['r','g','b'],[-0.1,0,0.1]):
    # for flas,isoFactor,color,xshift in zip([('n,pi+','n,pi+'),('n,pi+','p,pi0'),('p,pi0','p,pi0')],[1,-1*np.sqrt(2),2],['r','g','b'],[-0.1,0,0.1]):
        opa,opb=yu.getopab(pt,'l1',(occ,flas[0]),(occ,flas[1]))
        
        irow=0; icol=0
        dat=get2pt(opa,opb,yu.diags_loopless)
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)
        
        irow=0; icol=1
        dat=get2pt(opa,opb,yu.diags_all)
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)
        
ri='re'
# run2pt_iso_NpiNpi('0,0,0','N1pi1,a',prefactor=ri,ylim=[0,1e-10])
# run2pt_iso_NpiNpi('0,0,1','N1pi0,a',prefactor=ri,ylim=[0,1e-10])
# run2pt_iso_NpiNpi('0,0,1','N0pi1,a',prefactor=ri,ylim=[0,1e-10])

In [ ]:
def run2ptGEVP(pt,ofs,diags=yu.diags_all,title=None,ylim1=None,ylim2=None):
    fig, axs = yu.getFigAxs(1,2)
    title=title if title=='None' else 'pt={}; ofs={}'.format(pt,ofs)
    fig.suptitle(title)
    
    Nop=len(ofs)
    tmin=2; tmax0=15
    tRef=1
    
    for l,color,xshift in zip(['l1','l2'],['r','b'],[-0.1,0.1]):
        ops=yu.getops(pt,l,ofs)
        dat=get2ptMat(ops,diags)
        
        irow=0; icol=0
        def func(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0))
            return En.T
        mean,err,cov=yu.jackknife(dat,func)
        for ind in range(Nop):
            tmax=tmax0-3*ind
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','d','o'][ind])
            axs[irow,icol].set_xticks(range(0,tmax0,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=1
        for j in range(1):
            def func(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs)=yu.GEVP(t,tRef)
                t=eVecs[:,:,j].T
                t=t/t[0,:]
                return np.abs(t)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(Nop):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','<','>'][ind])
                axs[irow,icol].set_xticks(range(0,tmax0,2))  
            axs[irow,icol].set_yscale('log')
            axs[irow,icol].set_ylim(ylim2)
            axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])

# run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','n,pi+'),('N1pi1,a','p,pi0')],ylim1=[0.3,1.1])        
# run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','12')],ylim1=[0.3,1.1])        
# run2ptGEVP('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')],ylim1=[0.3,1.1])   

In [ ]:
tfs=[10,12,14]

def run3pt(opa,opb,insert,prefactor,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1)
    title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    
    needsVEV=yu.needsVEV(opa,opb,insert) and 'N-j' in diags
    
    dat3pt=[get3pt(opa,opb,insert+'_'+str(tf),diags)[:,1:tf] for tf in tfs]
    datVEVj=[data['VEV']['j'][insert]] if needsVEV else []
    datVEV2pt=[get2pt(opa,opb,yu.diags_all)] if needsVEV else []
    dat=[dat3pt,datVEVj,datVEV2pt]

    def func(dat):
        t=yu.meanDeep(dat)
        t=[(t[0][i]-t[1][0]*t[2][0][tf]) for i,tf in enumerate(tfs)] if needsVEV else [t[0][i] for i,tf in enumerate(tfs)] 
        t=yu.prefactorDeep(t,prefactor)
        return t

    (mean,err,cov)=yu.jackknife(dat,func)

    irow=0; icol=0
    for i_tf,tf in enumerate(tfs):
        tMean=mean[i_tf];tErr=err[i_tf]
        # print(tMean)
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
    axs[irow,icol].set_xticks(range(-6,7,1))
    axs[irow,icol].set_ylim(ylim1)
    
# run3pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'id_j-',1,diags={'N-j'})

# run3pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'gt_j+',1,diags={'NJN'})
# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'gt_j+',1,diags={'NJN'})

# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l1',('N0pi1,a','n,pi+')),'gt_j+',1,diags=yu.diags_all)

In [ ]:
tfs=[10,12,14]

def run3pt2pt(opa,opb,insert,prefactor,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1)
    title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    
    needsVEV=yu.needsVEV(opa,opb,insert) and 'N-j' in diags
    
    dat2pt=[get2pt(opa,opa,yu.diags_all),get2pt(opb,opb,yu.diags_all)]
    dat3pt=[get3pt(opa,opb,insert+'_'+str(tf),diags)[:,1:tf] for tf in tfs]
    datVEVj=[data['VEV']['j'][insert]]
    dat=[dat3pt,dat2pt,datVEVj]

    def func(dat):
        t=yu.meanDeep(dat)
        t=[t[0][i]/np.sqrt(
            np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
            np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
            np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
        ) - (t[2][0] if needsVEV else 0) 
           for i,tf in enumerate(tfs)]
        t=yu.prefactorDeep(t,prefactor)
        return t

    (mean,err,cov)=yu.jackknife(dat,func)

    irow=0; icol=0
    for i_tf,tf in enumerate(tfs):
        tMean=mean[i_tf];tErr=err[i_tf]
        # print(tMean)
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
    axs[irow,icol].set_xticks(range(-6,7,1))
    axs[irow,icol].set_ylim(ylim1)
    
# run3pt2pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'id_j+',1,diags={'NJN'})

# run3pt2pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'gt_j+',1,diags={'NJN'})
# run3pt2pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'gt_j+',1,diags={'NJN'})

# run3pt2pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l1',('N0pi1,a','n,pi+')),'gt_j+',1,diags=yu.diags_all)

In [ ]:
tfs=[10,12,14]; tRef=1
def get_eVec(eVecs):
    t=eVecs[:,0,0]
    t=eVecs[:,:,0]/t[:,None]
    return np.mean(t[10:14+1,:],axis=0)

def run3pt2ptGEVP(ptofas,ptofbs,insert,prefactor,proj,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1)
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    proj=np.array([[1,0],[0,1]])/2 if proj=='P0' else np.array([[1,0],[0,-1]])/2 if proj=='Pz' else proj
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
            
        needsVEV=pta==ptb and 'N-j' in diags
        
        def t_get(tf):
            t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),insert+'_'+str(tf),diags)[:,1:tf]\
                for lb in ['l1','l2']] for la in ['l1','l2']]
            return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
        
        dat3pt=[t_get(tf) for tf in tfs]
        dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),yu.diags_all),
                get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),yu.diags_all),]
        datVEVj=[data['VEV']['j'][insert]]
        dat=[dat3pt,dat2pt,datVEVj]
        
        AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

        def func(dat):
            t=yu.meanDeep(dat)
            t3pt,t2pt,tVEVj=t
            
            (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
            (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
            (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
            (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
            
            eVec_al1=get_eVec(eVecs_al1)
            eVec_al2=get_eVec(eVecs_al2)
            eVec_bl1=get_eVec(eVecs_bl1)
            eVec_bl2=get_eVec(eVecs_bl2)
            
            t2pt_a=(np.einsum('i,tij,j->t',np.conj(eVec_al1),t2pt[0],eVec_al1)+
                    np.einsum('i,tij,j->t',np.conj(eVec_al2),t2pt[1],eVec_al2))/2
            t2pt_b=(np.einsum('i,tij,j->t',np.conj(eVec_bl1),t2pt[2],eVec_bl1)+
                    np.einsum('i,tij,j->t',np.conj(eVec_bl2),t2pt[3],eVec_bl2))/2
            
            eVec_a=np.array([eVec_al1,eVec_al2])
            eVec_b=np.array([eVec_bl1,eVec_bl2])
            
            t=[(
                np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
            )/np.sqrt(
                np.real(t2pt_a[tf])*np.real(t2pt_b[tf])*\
                np.real(t2pt_a[1:tf][::-1])/np.real(t2pt_a[1:tf])*\
                np.real(t2pt_b[1:tf])/np.real(t2pt_b[1:tf][::-1])
            ) - (tVEVj if needsVEV else 0) 
            for i,tf in enumerate(tfs)]
            
            t=yu.prefactorDeep(t,prefactor)
            return t
        
        (mean,err,cov)=yu.jackknife(dat,func)

        irow=0; icol=0
        for i_tf,tf in enumerate(tfs):
            tMean=mean[i_tf];tErr=err[i_tf]
            # print(tMean)
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r' if case=='noGEVP' else 'b',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
        axs[irow,icol].set_xticks(range(-6,7,1))
        axs[irow,icol].set_ylim(ylim1)
    
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_jLoopless)
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_all,ylim1=[9,22])

# run3pt2ptGEVP(('0,0,0',[('a','p')]),('0,0,1',[('a','p'),('N1p i0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
# run3pt2ptGEVP(('0,0,0',[('a','p')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-4.5,-2.5])
# run3pt2ptGEVP(('0,0,0',[('a','p')]),('0,0,1',[('a','p'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_all,ylim1=[-10,-4])

# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])

# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5_j-',1,'Pz',diags=yu.diags_all)

# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.5,0.8])